# Google Colab으로 진행했습니다~

In [10]:
## import
import os
import sys
sys.path.append(os.path.abspath("src"))
import torch
import torch.nn as nn
from Inception import Inception
from GoogLeNet import GoogLeNet  # GoogLeNet 모델 불러오기
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim


In [4]:
# Google Colab 런타임 환경 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("사용 중인 디바이스:", device)

사용 중인 디바이스: cuda


In [5]:
# 데이터 전처리 및 로드
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # GoogLeNet에 맞는 크기로 조정
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
# CIFAR-10 데이터셋 다운로드 및 로드
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


100%|██████████| 170M/170M [00:06<00:00, 28.3MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [7]:
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified


In [20]:
# GoogLeNet 모델 초기화
model = GoogLeNet(num_classes=10).to(device)

In [21]:
# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

## train

In [26]:
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()

    # 손실과 정확도를 저장할 리스트
    epoch_losses = []
    epoch_accuracies = []

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        print(f"Epoch [{epoch + 1}/{num_epochs}] 시작")

        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs, aux1, aux2 = model(inputs)

            # 손실 계산
            loss1 = criterion(outputs, labels)
            loss2 = criterion(aux1, labels)
            loss3 = criterion(aux2, labels)
            loss = loss1 + 0.3 * loss2 + 0.3 * loss3

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            if batch_idx % 10 == 0:
                print(f"Batch [{batch_idx + 1}/{len(train_loader)}] - Loss: {loss.item():.4f}")

        # 에포크별 평균 손실과 정확도 계산
        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total

        # 손실과 정확도 저장
        epoch_losses.append(epoch_loss)
        epoch_accuracies.append(epoch_accuracy)

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")
        print("=" * 50)

    print("학습 완료")

    # 손실과 정확도 리스트 반환
    return epoch_losses, epoch_accuracies


In [30]:
def plot_training(epoch_losses, epoch_accuracies):
    import matplotlib.pyplot as plt
    plt.figure(figsize=(12, 5))

    # 손실 그래프
    plt.subplot(1, 2, 1)
    plt.plot(epoch_losses, marker='o', label='Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()
    plt.grid()

    # 정확도 그래프
    plt.subplot(1, 2, 2)
    plt.plot(epoch_accuracies, marker='o', label='Accuracy', color='orange')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training Accuracy')
    plt.legend()
    plt.grid()

    plt.show()


In [ ]:
# 학습 실행
num_epochs = 10
epoch_losses, epoch_accuracies = train(model, train_loader, criterion, optimizer, num_epochs)

# 그래프 출력
plot_training(epoch_losses, epoch_accuracies)


In [ ]:
# train 검증
epoch_losses, epoch_accuracies = train(model, train_loader, criterion, optimizer, num_epochs)
print("손실 기록:", epoch_losses)
print("정확도 기록:", epoch_accuracies)


## test

In [32]:
print(f"Test dataset size: {len(test_loader.dataset)}")


Test dataset size: 10000


In [33]:
# 테스트
import matplotlib.pyplot as plt

def test(model, test_loader, criterion):
    model.eval()  # 평가 모드
    test_loss = 0.0
    correct = 0
    total = 0
    batch_losses = []  # 배치별 손실 저장

    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            # 모델 출력
            outputs = model(inputs)

            # 손실 계산
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            batch_losses.append(loss.item())  # 배치 손실 저장

            # 정확도 계산
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    # 평균 손실 및 정확도 계산
    avg_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")
    return avg_loss, accuracy, batch_losses


In [34]:
def plot_test_results(batch_losses, test_accuracy):
    plt.figure(figsize=(12, 6))

    # 배치별 손실 그래프
    plt.subplot(1, 2, 1)
    plt.plot(batch_losses, marker='o', label='Batch Loss')
    plt.title('Batch Loss During Testing')
    plt.xlabel('Batch Index')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()

    # 테스트 정확도 표시
    plt.subplot(1, 2, 2)
    plt.bar(['Test Accuracy'], [test_accuracy], color='orange')
    plt.title('Test Accuracy')
    plt.ylim(0, 100)
    plt.ylabel('Accuracy (%)')
    plt.grid(axis='y')

    plt.tight_layout()
    plt.show()


In [ ]:
# 테스트 실행
test_loss, test_accuracy, batch_losses = test(model, test_loader, criterion)

# 테스트 그래프 출력
plot_test_results(batch_losses, test_accuracy)
